# Cognitive Library Explorer V6 — Notebook実行例

このノートブックは **Streamlit を起動せず**に、同梱の `LibraryAnalyzerV6`（AST解析）でライブラリを解析し、`nodes / edges / errors` を確認するためのサンプルです。

- 解析対象は `lib_name` で指定します（例: `pandas` / `numpy` など）
- 解析は **import実行を最小化**しつつ、`.py` をAST(抽象構文木)で走査します（副作用が起きにくい設計）


In [ ]:
# セットアップ: このリポジトリの src を import パスへ追加
import sys
from pathlib import Path

ROOT = Path().resolve()  # Notebookを「プロジェクト直下」で開く想定
SRC = ROOT / "src"
if SRC.exists() and str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

print("sys.path[0] =", sys.path[0])

In [ ]:
# 解析対象ライブラリ名（distribution名でもOKな場合があります）
lib_name = "pandas"  # 例: "pandas", "numpy", "requests", "scikit-learn" など


In [ ]:
from v6.core.analyzer_v6 import LibraryAnalyzerV6

analyzer = LibraryAnalyzerV6(lib_name, max_files=1500)  # 大規模なら max_files を下げると軽くなります
summary, nodes, edges, errors = analyzer.analyze()

summary

In [ ]:
import pandas as pd

df_nodes = pd.DataFrame(nodes)
df_edges = pd.DataFrame(edges)
df_errors = pd.DataFrame(errors)

print("nodes:", df_nodes.shape, "edges:", df_edges.shape, "errors:", df_errors.shape)
df_nodes.head(10)

In [ ]:
# Type別の件数
if not df_nodes.empty and "Type" in df_nodes.columns:
    display(df_nodes["Type"].value_counts())

In [ ]:
# 追加の加工（分類 / 引数対応表など）
from v6.core.classify_v6 import enrich_with_classification
from v6.core.param_map_v6 import build_param_map_tables

df_nodes2 = enrich_with_classification(df_nodes)
param_tables = build_param_map_tables(df_nodes2)

display(df_nodes2.head(10))
display(param_tables["ParamOverview"].head(20))

In [ ]:
# （任意）CSVとして保存
out_dir = Path("out")
out_dir.mkdir(exist_ok=True)

df_nodes2.to_csv(out_dir / f"{lib_name}_nodes.csv", index=False, encoding="utf-8-sig")
df_edges.to_csv(out_dir / f"{lib_name}_edges.csv", index=False, encoding="utf-8-sig")
df_errors.to_csv(out_dir / f"{lib_name}_errors.csv", index=False, encoding="utf-8-sig")

print("saved to:", out_dir.resolve())